In [1]:
import api
import pandas as pd
import numpy as np

In [52]:
#vsi trgovalni pari borzi Bitstamp
pari = ['btcusd', 'btceur', 'eurusd', 'xrpusd', 'xrpeur', 'xrpbtc', 'ltcusd', 'ltceur', 'ltcbtc', 'ethusd', 'etheur', 'ethbtc']
for par in pari:
    api.zajemi(par)

In [108]:
def pametno_preberi(par):
    ime = par + '.csv'
    data = pd.read_csv(ime)
    data['prva'] = ime[:3].upper()
    data['druga'] = ime[3:-4].upper()
    return data

In [109]:
df = pametno_preberi('btcusd')
df.head(3)

,date,tid,price,amount,type,prva,druga
0,1503082472,18776518,4154.99,0.086100,0,BTC,USD
1,1503082457,18776513,4145.52,0.060000,1,BTC,USD
2,1503082448,18776507,4160.70,0.013677,0,BTC,USD


In [110]:
len(df)

23894

Pri type 0 pomeni 'buy' in 1 pomeni 'sell'.

In [111]:
pari = ['btcusd', 'btceur', 'eurusd', 'xrpusd', 'xrpeur', 'xrpbtc', 'ltcusd', 'ltceur', 'ltcbtc', 'ethusd', 'etheur', 'ethbtc']
pari = pari[1:] #btcusd sem že dal v df
for par in pari:
    print(par)
    tmp = pametno_preberi(par)
    df = df.append(tmp)

btceur
eurusd
xrpusd
xrpeur
xrpbtc
ltcusd
ltceur
ltcbtc
ethusd
etheur
ethbtc


In [112]:
#Število vseh transakcij na Bitstampu v zadnjih 24 urah:
len(df)

56209

In [113]:
pd.options.mode.chained_assignment = None  # default='warn'
df = df.drop(df.columns[[0, 1]], axis=1)
# priredim še enotni stolpec cen, kjer bodo vse cene v dolarjih 
df['ceneDolarji'] = 0
df['ceneDolarji'][df['druga'] == 'USD'] = df['price']
df['ceneDolarji'][df['druga'] == 'EUR'] = df['price'] * 1.17 #eur-usd tečaj 18.8.2017
df['ceneDolarji'][df['druga'] == 'BTC'] = df['price'] * 4100 #btc-usd tečaj 18.8.2017, pazi velika volatilnost!
df.head(3)

,price,amount,type,prva,druga,ceneDolarji
0,4154.99,0.086100,0,BTC,USD,4154.99
1,4145.52,0.060000,1,BTC,USD,4145.52
2,4160.70,0.013677,0,BTC,USD,4160.70


Pomen na grafu: leva ----> desna

In [105]:
df.druga.unique()

array(['USD', 'EUR', 'BTC'], dtype=object)

In [114]:
#informacije iz stolpcev 'type', 'prva' in 'druga', pretvorim v le dva stolpca
df['Leva'] = np.where(df['type']==0, df['druga'], df['prva'])
df['Desna'] = np.where(df['type']==0, df['prva'], df['druga'])
df['combo'] = df['Leva'] + df['Desna']

,price,amount,type,prva,druga,ceneDolarji,Leva,Desna,combo
0,4154.99,0.086100,0,BTC,USD,4154.99,USD,BTC,USDBTC
1,4145.52,0.060000,1,BTC,USD,4145.52,BTC,USD,BTCUSD
2,4160.70,0.013677,0,BTC,USD,4160.70,USD,BTC,USDBTC
3,4160.72,0.060285,0,BTC,USD,4160.72,USD,BTC,USDBTC
4,4160.72,0.295144,1,BTC,USD,4160.72,BTC,USD,BTCUSD


In [115]:
df = df.drop(df.columns[[2, 3, 4]], axis=1)
df.head()

,price,amount,ceneDolarji,Leva,Desna,combo
0,4154.99,0.086100,4154.99,USD,BTC,USDBTC
1,4145.52,0.060000,4145.52,BTC,USD,BTCUSD
2,4160.70,0.013677,4160.70,USD,BTC,USDBTC
3,4160.72,0.060285,4160.72,USD,BTC,USDBTC
4,4160.72,0.295144,4160.72,BTC,USD,BTCUSD


In [125]:
df['volume'] = df['ceneDolarji'] * df['amount']

In [136]:
df_combo = df.groupby('combo')
df_combo.sum()['amount']
df_combo.mean()['ceneDolarji']
df_combo.count()['price']
df_combo.sum()['volume']

combo
BTCETH    8.176651e+05
BTCEUR    3.409430e+06
BTCLTC    1.679863e+05
BTCUSD    2.678484e+07
BTCXRP    9.461534e+05
ETHBTC    8.362155e+05
ETHEUR    7.307879e+05
ETHUSD    1.156005e+06
EURBTC    4.086926e+06
EURETH    5.174487e+05
EURLTC    2.170484e+05
EURUSD    1.113132e+06
EURXRP    3.170422e+05
LTCBTC    2.938311e+05
LTCEUR    4.622626e+05
LTCUSD    7.154777e+05
USDBTC    3.247411e+07
USDETH    1.285384e+06
USDEUR    2.963136e+05
USDLTC    1.361366e+06
USDXRP    1.244216e+06
XRPBTC    7.433316e+05
XRPEUR    5.104041e+05
XRPUSD    1.283383e+06
Name: volume, dtype: float64

Ni uredu, ker ni 24 parov (sedaj jih je samo 18).

In [145]:
#zelo grd način, sigruno se da lepše :)
tmp = pd.DataFrame(df_combo.sum()['volume'])
tmp.index.name = None
tmp['combo'] = tmp.index
tmp2 = pd.DataFrame(df_combo.sum()['amount'])
tmp2.index.name = None
tmp2['combo'] = tmp2.index
tmp3 = pd.DataFrame(df_combo.mean()['ceneDolarji'])
tmp3.index.name = None
tmp3['combo'] = tmp3.index
tmp4 = pd.DataFrame(df_combo.count()['price'])
tmp4.index.name = None
tmp4['combo'] = tmp4.index
tmp4

,price,combo
BTCETH,1084,BTCETH
BTCEUR,2652,BTCEUR
BTCLTC,346,BTCLTC
BTCUSD,8812,BTCUSD
BTCXRP,1451,BTCXRP
ETHBTC,1110,ETHBTC
ETHEUR,1503,ETHEUR
ETHUSD,1555,ETHUSD
EURBTC,4376,EURBTC
EURETH,1232,EURETH


In [155]:
final = pd.merge(tmp, tmp2, on='combo')
final = pd.merge(final, tmp3, on='combo')
final = pd.merge(final, tmp4, on='combo')
#final = final.join(tmp3, on='combo', how='left', lsuffix='_left', rsuffix='_right')
#final = final.join(tmp4, on='combo', how='left', lsuffix='_left', rsuffix='_right')
final.columns = ['volume', 'combo', 'nr_tokens', 'priceUSD', 'nr_txs']
final

,volume,combo,nr_tokens,priceUSD,nr_txs
0,8.176651e+05,BTCETH,2.820694e+03,289.595394,1084
1,3.409430e+06,BTCEUR,8.015830e+02,4268.960374,2652
2,1.679863e+05,BTCLTC,3.837905e+03,43.586521,346
3,2.678484e+07,BTCUSD,6.309885e+03,4250.951080,8812
4,9.461534e+05,BTCXRP,6.063312e+06,0.153744,1451
5,8.362155e+05,ETHBTC,2.896598e+03,288.915985,1110
6,7.307879e+05,ETHEUR,2.424987e+03,301.813044,1503
7,1.156005e+06,ETHUSD,3.846396e+03,300.319717,1555
8,4.086926e+06,EURBTC,9.554252e+02,4280.375158,4376
9,5.174487e+05,EURETH,1.711327e+03,302.799913,1232


In [157]:
final['prva'] = final['combo'].str.extract('(^...)', expand=True)
final['druga'] = final['combo'].str.extract('(...$)', expand=True)
final

,volume,combo,nr_tokens,priceUSD,nr_txs,prva,druga
0,8.176651e+05,BTCETH,2.820694e+03,289.595394,1084,BTC,ETH
1,3.409430e+06,BTCEUR,8.015830e+02,4268.960374,2652,BTC,EUR
2,1.679863e+05,BTCLTC,3.837905e+03,43.586521,346,BTC,LTC
3,2.678484e+07,BTCUSD,6.309885e+03,4250.951080,8812,BTC,USD
4,9.461534e+05,BTCXRP,6.063312e+06,0.153744,1451,BTC,XRP
5,8.362155e+05,ETHBTC,2.896598e+03,288.915985,1110,ETH,BTC
6,7.307879e+05,ETHEUR,2.424987e+03,301.813044,1503,ETH,EUR
7,1.156005e+06,ETHUSD,3.846396e+03,300.319717,1555,ETH,USD
8,4.086926e+06,EURBTC,9.554252e+02,4280.375158,4376,EUR,BTC
9,5.174487e+05,EURETH,1.711327e+03,302.799913,1232,EUR,ETH


In [158]:
final.to_csv('tidy.csv', index=False)

#### ORDER BOOK test data:

In [7]:
#fajl s transkacijami sem "ročno" preimenoval v btcusd_txs.csv
api.zajemi('btcusd', 'hour')
api.zajemi_order('btcusd')

'Par btcusd order zajet.'

In [8]:
order = pd.read_csv('btcusd_order.csv')
order.head()

,price,amount,type
0,4714.46,2.947000,0
1,4714.45,0.210000,0
2,4712.24,2.119800,0
3,4712.02,1.189207,0
4,4711.00,0.001120,0


In [9]:
txs = pd.read_csv('btcusd_txs.csv')
txs.head()

,date,tid,price,amount,type
0,1504190875,19728467,4718.69,0.162841,0
1,1504190869,19728464,4714.46,0.159000,1
2,1504190869,19728463,4714.46,0.211000,1
3,1504190869,19728462,4714.46,0.210000,1
4,1504190869,19728461,4714.46,0.210000,1


In [10]:
len(order)

4654

In [11]:
len(txs)

1080

Kraken: https://www.kraken.com/en-gb/help/api#get-order-book

Kraken trade info (povsem isto kot pri Bitstampu): https://www.kraken.com/en-gb/help/api#get-recent-trades